In [33]:
# First Env. Download Data
!pip install roboflow

from roboflow import Roboflow
import boto3
import os
from tqdm.notebook import tqdm

prefix = "fire-dataset-cl-1"

Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 33.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting un

In [26]:
!ls -lh
# !unzip -o fire_dataset/

total 388M
-rw-r--r-- 1 sagemaker-user users 388M Jun 13 08:53 archive.zip
drwxr-xr-x 4 sagemaker-user users   48 Jun 13 08:55 fire_dataset
-rw-r--r-- 1 sagemaker-user users 2.7K Jun 13 09:02 mj-first.ipynb


In [29]:
bucket = "fire-project-hs"


def create_bucket(bucket_name):
    session = boto3.Session()

    s3_client = session.client('s3')
    region = session.region_name

    # Create the bucket
    if region == 'us-east-1':
        # Us-east-1 does not require a LocationConstraint
        s3_client.create_bucket(Bucket=bucket_name)
    else:
        s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )

    print(f"Bucket {bucket_name} created in region {region}")


create_bucket(bucket)

Bucket fire-project-hs created in region eu-central-1


In [32]:
def upload_folder_to_s3(bucket_name, prefix, local_folder):
    s3_resource = boto3.Session().resource('s3')
    bucket = s3_resource.Bucket(bucket_name)

    for root, dirs, files in os.walk(local_folder):
        for file in tqdm(files, root):
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_folder)
            s3_path = os.path.join(prefix, relative_path).replace("\\", "/")  # Ensure S3 uses forward slashes

            bucket.Object(s3_path).upload_file(local_path)


local_folder = 'fire_dataset'
upload_folder_to_s3(bucket, prefix, local_folder)

fire_dataset: 0it [00:00, ?it/s]

fire_dataset/fire_images:   0%|          | 0/755 [00:00<?, ?it/s]

fire_dataset/non_fire_images:   0%|          | 0/244 [00:00<?, ?it/s]

In [34]:
# ------------------------------
rf = Roboflow(api_key="4R0rPPW6Fi14nje1oLu5")
project = rf.workspace("fire-iak32").project("fire-project")
version = project.version(4)
dataset = version.download("multiclass")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fire-project-4 in multiclass:: 100%|██████████| 486/486 [00:00<00:00, 3861.60it/s]


In [37]:
# !ls fire-project-4/train

local_folder = 'fire-project-4'
upload_folder_to_s3(bucket, prefix, local_folder)

fire-project-4:   0%|          | 0/2 [00:00<?, ?it/s]

fire-project-4/train:   0%|          | 0/483 [00:00<?, ?it/s]

In [38]:
# ----------------------------------
project = rf.workspace("fire-dataset-tp9jt").project("fire-detection-sejra")
version = project.version(1)
dataset = version.download("multiclass")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Fire-Detection-1 in multiclass:: 100%|██████████| 8943/8943 [00:01<00:00, 5749.62it/s]


In [40]:
local_folder = 'Fire-Detection-1'
upload_folder_to_s3(bucket, prefix, local_folder)

Fire-Detection-1:   0%|          | 0/2 [00:00<?, ?it/s]

Fire-Detection-1/train:   0%|          | 0/8940 [00:00<?, ?it/s]